In [2]:
instruments = broker.kite.instruments("NSE")
matches = [i for i in instruments if "HDFC" in i["tradingsymbol"]]
for m in matches:
    print(m["tradingsymbol"], m["instrument_token"])

HDFCLIFE 119553
HDFCBANK 341249
HDFCAMC 1086465
HDFCNEXT50 2718465
HDFCNIF100 2722049
HDFCSILVER 2784257
HDFCGROWTH 2877697
HDFCQUAL 2881281
HDFCVALUE 2882561
HDFCMOMENT 2953729
HDFCLOWVOL 2956033
HDFCNIFTY 2967297
HDFCSENSEX 2967809
HDFCNIFIT 3097857
HDFCPVTBAN 3099649
HDFCBSE500 3642881
HDFCSML250 3643649
HDFCMID150 3644417
HDFCLIQUID 4679425
HDFCGOLD 5003009
771HDFCB33-N0 5312769
HDFCNIFBAN 5742849
HDFCPSUBK 5784321
765HDFC34-N1 5944833


## Data log of entries, exits and P&L

In [2]:
import pandas as pd

# bt_df is the DataFrame returned by backtest()
contract_size = 1  # for equity

# Make sure bt_df is indexed by the `date` column:
df = bt_df.set_index("date")

# Extract entry & exit rows
entries = df[df.signal.isin(["BUY","SELL"])].copy()
exits   = df[df.signal == "EXIT"].copy()

trades = []
# Iterate over matching entry/exit pairs
for (entry_time, ent_row), (exit_time, ex_row) in zip(entries.iterrows(), exits.iterrows()):
    side        = ent_row.signal
    entry_price = ent_row.close
    exit_price  = ex_row.close
    qty         = int(ent_row.qty)
    notional    = entry_price * qty * contract_size
    pnl         = float(ex_row.pnl)
    duration    = exit_time - entry_time

    trades.append({
        "side":        side,
        "entry_time":  entry_time,   # now a Timestamp
        "exit_time":   exit_time,    # now a Timestamp
        "entry_price": entry_price,
        "exit_price":  exit_price,
        "qty":         qty,
        "notional":    notional,
        "pnl":         pnl,
        "duration":    duration,
    })

report = pd.DataFrame(trades)
print(report.to_string(index=False))
print(f"\nTOTAL PnL: ₹{report.pnl.sum():.2f}")

side                entry_time                 exit_time  entry_price  exit_price  qty  notional    pnl        duration
 BUY 2025-04-25 15:00:00+05:30 2025-04-28 13:15:00+05:30       1303.6      1368.9    7    9125.2  457.1 2 days 22:15:00
 BUY 2025-04-28 14:20:00+05:30 2025-05-05 09:40:00+05:30       1366.1      1438.4    7    9562.7  506.1 6 days 19:20:00
 BUY 2025-05-05 10:10:00+05:30 2025-05-06 11:15:00+05:30       1435.5      1412.0    6    8613.0 -141.0 1 days 01:05:00
SELL 2025-05-08 09:50:00+05:30 2025-05-12 13:40:00+05:30       1409.8      1431.0    7    9868.6 -148.4 4 days 03:50:00
 BUY 2025-05-13 09:20:00+05:30 2025-05-22 09:45:00+05:30       1431.0      1408.4    6    8586.0 -135.6 9 days 00:25:00
 BUY 2025-05-23 11:10:00+05:30 2025-06-02 09:15:00+05:30       1430.5      1401.1    6    8583.0 -176.4 9 days 22:05:00
SELL 2025-06-02 09:40:00+05:30 2025-06-03 10:20:00+05:30       1400.5      1422.5    7    9803.5 -154.0 1 days 00:40:00
 BUY 2025-06-05 11:20:00+05:30 2025-06-1

In [1]:
# 1️⃣ Imports
from algo.broker       import KiteWrapper
from algo.config       import load_config
from algo.features     import add_indicators
from algo.model        import load_or_train
from algo.backtester   import backtest
import pandas as pd

# 2️⃣ Parameters
SYMBOL     = "HDFCBANK"
INTERVAL   = "3minute"
TRAIN_DAYS = 180
TEST_DAYS  = 20

CAPITAL    = 100_000
SL_PCT     = 0.0015
TP_PCT     = 0.0025
TRAIL_PCT  = 0.0020
HOLD_MAX   = 15

UPPER_PROB = 0.62
LOWER_PROB = 0.32

# 3️⃣ Fetch & prepare training data
cfg        = load_config()
broker     = KiteWrapper(cfg)

hist_train = broker.history(
    days=TRAIN_DAYS,
    interval=INTERVAL,
    tradingsymbol=SYMBOL
)
df_train = add_indicators(hist_train).ffill()

# 4️⃣ Load or retrain model (retrain=False will reuse your new pipeline)
model = load_or_train(df_train, retrain=False)

# 5️⃣ Fetch & prepare test slice
hist_test = broker.history(
    days=TEST_DAYS,
    interval=INTERVAL,
    tradingsymbol=SYMBOL
)
df_test = add_indicators(hist_test).ffill()

# 6️⃣ Run backtest
trades, metrics = backtest(
    df_test,
    model=model,
    capital=CAPITAL,
    contract_size=1,
    sl_pct=SL_PCT,
    tp_pct=TP_PCT,
    trail_pct=TRAIL_PCT,
    hold_max=HOLD_MAX,
    upper=UPPER_PROB,
    lower=LOWER_PROB,
)

# 7️⃣ Review results
print("Back-test metrics:", metrics)
print("\nFirst few trades:\n", trades.head())


Back-test metrics: {'Trades': 24.0, 'WinRate': 0.375, 'PnL': 800.9007782652479, 'EquityFinal': 100800.90077826525}

First few trades:
                            side   price         pnl        equity reason
ts                                                                      
2025-06-09 13:06:00+05:30   BUY  1979.6   -0.593880  99999.406120       
2025-06-09 13:51:00+05:30  EXIT  1979.4  -10.696871  99988.709249   TIME
2025-06-09 14:00:00+05:30   BUY  1980.1   -0.594030  99988.115219       
2025-06-09 14:24:00+05:30  EXIT  1975.9 -212.703269  99775.411949     SL
2025-06-09 14:33:00+05:30   BUY  1978.7   -0.593610  99774.818339       


In [ ]:
import time
import random
import itertools
import pandas as pd
from tqdm import tqdm

from algo.broker     import KiteWrapper
from algo.config     import load_config
from algo.features   import add_indicators
from algo.model      import _prepare_xy
from algo.backtester import backtest

from sklearn.pipeline        import Pipeline
from sklearn.impute          import SimpleImputer
from sklearn.preprocessing   import StandardScaler
from sklearn.ensemble        import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

# ─── 1) Fetch & feature-engineer ───────────────────────────────
cfg    = load_config()
broker = KiteWrapper(cfg)
hist   = broker.history(days=200, interval="3minute", tradingsymbol="HDFCBANK")
df     = add_indicators(hist).ffill()

# split off final TEST_DAYS for out-of-sample
TEST_DAYS = 60
df_train, df_test = df.iloc[:-TEST_DAYS], df.iloc[-TEST_DAYS:]

# prepare X/y for ML
X_all, y_all = _prepare_xy(df_train)
Xtr, Xv, ytr, yv = train_test_split(X_all, y_all, test_size=0.2, shuffle=False)

# ─── 2) Define search space ────────────────────────────────────
ml_grid = {
    "n_estimators":  [50, 100, 200],
    "max_depth":     [3, 5, 7],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample":     [0.6, 0.8, 1.0],
}

strat_grid = {
    "sl_pct":    [0.0005, 0.001, 0.0015, 0.002],
    "tp_pct":    [0.001, 0.002, 0.0025, 0.003],
    "trail_pct": [0.001, 0.0015, 0.002, 0.0025],
    "hold_max":  [5, 10, 15, 20],
    "upper":     [0.55, 0.6, 0.65, 0.7],
    "lower":     [0.3, 0.35, 0.4, 0.45],
}

# build full list of all possible combos
all_combos = list(itertools.product(
    ml_grid["n_estimators"],
    ml_grid["max_depth"],
    ml_grid["learning_rate"],
    ml_grid["subsample"],
    strat_grid["sl_pct"],
    strat_grid["tp_pct"],
    strat_grid["trail_pct"],
    strat_grid["hold_max"],
    strat_grid["upper"],
    strat_grid["lower"],
))

# randomly sample N_SAMPLES combos
N_SAMPLES = 50
sampled = random.sample(all_combos, k=min(N_SAMPLES, len(all_combos)))

def make_pipe(params):
    return Pipeline([
        ("imputer", SimpleImputer()),
        ("scaler",  StandardScaler()),
        ("gb",      GradientBoostingClassifier(**params)),
    ])

# ─── 3) Search loop with progress bar ───────────────────────────
results = []
start = time.time()

for (n_est, depth, lr, subs,
     sl_pct, tp_pct, trail_pct,
     hold_max, upper, lower) in tqdm(
    sampled,
    total=len(sampled),
    desc="Hyper-search",
    unit="combo"
):
    # ① train a fresh GB model
    gb_params = {
        "n_estimators":  n_est,
        "max_depth":     depth,
        "learning_rate": lr,
        "subsample":     subs,
    }
    pipe = make_pipe(gb_params).fit(Xtr, ytr)

    # ② backtest with these strategy settings
    try:
        _, m = backtest(
            df_test,
            model        = pipe,
            capital      = 100_000,
            contract_size= 1,
            sl_pct       = sl_pct,
            tp_pct       = tp_pct,
            trail_pct    = trail_pct,
            hold_max     = hold_max,
            upper        = upper,
            lower        = lower,
        )
        pnl, wr, trades = m["PnL"], m["Win Rate"], m["Trades"]
    except KeyError:
        pnl, wr, trades = 0.0, 0.0, 0

    # record results
    results.append({
        **gb_params,
        "sl_pct":    sl_pct,
        "tp_pct":    tp_pct,
        "trail_pct": trail_pct,
        "hold_max":  hold_max,
        "upper":     upper,
        "lower":     lower,
        "PnL":       pnl,
        "Win Rate":  wr,
        "Trades":    trades,
    })

elapsed = time.time() - start
print(f"\nSearched {len(sampled)} combos in {elapsed:.0f}s")

# ─── 4) Collect & inspect ───────────────────────────────────────
df_res = pd.DataFrame(results)
# optionally drop combos that never traded:
df_res = df_res[df_res["Trades"] > 0]

if not df_res.empty:
    best = df_res.sort_values("PnL", ascending=False).iloc[0]
    print("\n🏆 Best combo by PnL:\n", best)
else:
    print("\n⚠️ No trades fired on any combo. Try widening your thresholds.")

print("\nAll results:\n", df_res.sort_values("PnL", ascending=False))


Hyper-search:   0%|          | 0/50 [00:00<?, ?combo/s]C:\Users\sreej\projects\zerodha-bot\algo\features.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = out if out is not None else pd.NA
C:\Users\sreej\projects\zerodha-bot\algo\features.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["atr"] = atr if atr is not None else pd.NA
C:\Users\sreej\projects\zerodha-bot\algo\features.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l